In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DataTypes, StructField}
import org.apache.spark.sql.types._

import java.util.Date
import java.time.format.DateTimeFormatter
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.{functions => T}
import org.apache.spark.sql.expressions
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DataTypes, StructField}
import org.apache.spark.sql.types._
import java.util.Date
import java.time.format.DateTimeFormatter
import org.apache.spark.sql.{functions=>F}
import org.apache.spark.sql.{functions=>T}
import org.apache.spark.sql.expressions
import org.apache.spark.sql.expressions.Window


In [11]:
// создаем spark сессию
val spark:SparkSession = SparkSession.builder()
      .master("local[1]")
      .appName("Task #4")
      .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@100f2a56


In [12]:
// импортирование дополнительных пакетов
import spark.implicits._

import spark.implicits._


In [13]:
// Даты, на которые будет строиться витрина
val pDate1 = "2020-11-01"
val pDate2 = "2020-11-02"
val pDate3 = "2020-11-03"
val pDate4 = "2020-11-04"

// Общий параметр для запуска витрин
// Для запуска витрин просто пишем: val pDateCommon = pDate1 или val pDateCommon = pDate2 и.т.д

val pDateCommon = pDate1 // Все витрины будут построены на дату pDate2

pDate1: String = 2020-11-01
pDate2: String = 2020-11-02
pDate3: String = 2020-11-03
pDate4: String = 2020-11-04
pDateCommon: String = 2020-11-01


In [14]:
// Определим схему данных для таблицы клиентов
val schemaClients = DataTypes.createStructType(Array[StructField](
      DataTypes.createStructField("ClientId", DataTypes.IntegerType, false),
      DataTypes.createStructField("ClientName", DataTypes.StringType, false),
      DataTypes.createStructField("Type", DataTypes.StringType, true),
      DataTypes.createStructField("Form", DataTypes.StringType, true),  
      DataTypes.createStructField("RegisterDate", DataTypes.DateType, false)))

// Определим схему данных для таблицы счетов
val schemaAccounts = StructType(Array[StructField](
      DataTypes.createStructField("AccountId", DataTypes.IntegerType, false),
      DataTypes.createStructField("AccountNum", DataTypes.StringType, false),
      DataTypes.createStructField("ClientId", DataTypes.IntegerType, false),
      DataTypes.createStructField("DateOpen", DataTypes.DateType, false)))

// Определим схему данных для таблицы банковских операций 
val schemaOperations = StructType(Array(
      StructField("AccountDB", IntegerType, false),
      StructField("AccountCR", IntegerType, false),
      StructField("DateOp", DateType, false),
      StructField("Amount", DecimalType(10, 2), false),
      StructField("Currency",StringType, false),  
      StructField("Comment", StringType, true)))
                                                
// Определим схему данных для таблицы курсов валют
val schemaRates = DataTypes.createStructType(Array[StructField](
      DataTypes.createStructField("Currency", DataTypes.StringType, false),
      DataTypes.createStructField("Rate", DecimalType(10, 2), false),
      DataTypes.createStructField("RateDate", DataTypes.DateType, false)))

schemaClients: org.apache.spark.sql.types.StructType = StructType(StructField(ClientId,IntegerType,false), StructField(ClientName,StringType,false), StructField(Type,StringType,true), StructField(Form,StringType,true), StructField(RegisterDate,DateType,false))
schemaAccounts: org.apache.spark.sql.types.StructType = StructType(StructField(AccountId,IntegerType,false), StructField(AccountNum,StringType,false), StructField(ClientId,IntegerType,false), StructField(DateOpen,DateType,false))
schemaOperations: org.apache.spark.sql.types.StructType = StructType(StructField(AccountDB,IntegerType,false), StructField(AccountCR,IntegerType,false), StructField(DateOp,DateType,false), StructField(Amount,DecimalType(10,2),false), StructField(Currency,StringType,false), StructField(Comment,StringType...


In [15]:
// чтение полного набора данных
// считаем csv файл клиентов Clients.csv, запишем прочитанное в dataframe
val dfClients = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("dateformat","yyyy-MM-dd")
        .option("enforceSchema", true)
        .schema(schemaClients)
        .load("files/Clients.csv")

// считаем csv файл счетов Account.csv, запишем прочитанное в dataframe
val dfAccounts = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("dateformat","yyyy-MM-dd")
        .option("enforceSchema", true)
        .schema(schemaAccounts)
        .load("files/Account.csv")

// считаем csv файл банковских операций Operation.csv, запишем прочитанное в dataframe
val dfOperations = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("enforceSchema", true)
        .option("dateformat","yyyy-MM-dd")
        .schema(schemaOperations)
        .load("files/Operation.csv")

// считаем csv файл курса валют Rate.csv, запишем прочитанное в dataframe
val dfRates = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("enforceSchema", true)
        .schema(schemaRates)
      .load("files/Rate.csv")



dfClients: org.apache.spark.sql.DataFrame = [ClientId: int, ClientName: string ... 3 more fields]
dfAccounts: org.apache.spark.sql.DataFrame = [AccountId: int, AccountNum: string ... 2 more fields]
dfOperations: org.apache.spark.sql.DataFrame = [AccountDB: int, AccountCR: int ... 4 more fields]
dfRates: org.apache.spark.sql.DataFrame = [Currency: string, Rate: decimal(10,2) ... 1 more field]


In [7]:
// Считаем строки из БД postgres
// СУБД postgres находится локально в докере

val masks = spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://localhost:5435/bank")
    .option("dbtable", "list")
    .option("user", "postgres")
    .option("password", "postgres")
    .option("driver", "org.postgresql.Driver")
    .load()

masks.select("maskvalue").filter("id = 1").show()

masks: org.apache.spark.sql.DataFrame = [id: int, maskvalue: string]


+---------+
|maskvalue|
+---------+
|        %|
+---------+



<console>: 25: error: not found: value Rates

In [9]:
clientsAndAccounts.orderBy("ClientId").count()

<console>: 26: error: not found: value clientsAndAccounts

<console>: 24: error: not found: value dfOperations